# Query Model Repository for best node embeddings model

In [1]:
from hops import model
from hops.model import Metric
MODEL_NAME="NodeEmbeddings"
EVALUATION_METRIC="accuracy"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
56,application_1607211657348_0058,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [3]:
best_model['experimentId']

'application_1607211657348_0057_2'

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [4]:
import tensorflow as tf
from tensorflow import keras  

import pandas as pd
from stellargraph import StellarDiGraph
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.data import UnsupervisedSampler, BiasedRandomWalk
from stellargraph.layer import Node2Vec
import pydoop.hdfs as pydoop
import hsfs

In [5]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [6]:
node_td = fs.get_training_dataset("node_td", 1)
edge_td = fs.get_training_dataset("edges_td", 1)

In [7]:
# Get fg as pandas
node_pdf = node_td.read().toPandas()
edge_pdf = edge_td.read().drop("tran_timestamp").toPandas()

In [8]:
node_data = pd.DataFrame(node_pdf[['tx_behavior_id','prior_sar','initial_deposit']], index=node_pdf['id'])

print('Defining StellarDiGraph')
G =StellarDiGraph(node_data,
                      edges=edge_pdf, 
                      edge_type_column="tx_type")


Defining StellarDiGraph

In [9]:
walk_number = 2
walk_length = 2
batch_size = 1
emb_size = 16
# Extracting node embeddings
walker = BiasedRandomWalk(
        G,
        n=walk_number,
        length=walk_length,
        p=0.5,  # defines probability, 1/p, of returning to source node
        q=2.0,  # defines probability, 1/q, for moving to a node away from the source node
    )
unsupervised_samples = UnsupervisedSampler(G, nodes=list(G.nodes()), walker=walker)
generator = Node2VecLinkGenerator(G, batch_size)

node2vec = Node2Vec(emb_size, generator=generator)
x_inp, x_out = node2vec.in_out_tensors()

x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [10]:
latest = tf.train.latest_checkpoint("hdfs:///Projects/amlsim2/Experiments/" + best_model['experimentId'])
embedding_model.load_weights(latest)

In [11]:
nodes = list(G.nodes())
node_gen = Node2VecNodeGenerator(G, batch_size).flow(nodes)
node_embeddings = embedding_model.predict(node_gen)

In [12]:
node_embeddings[0].tolist()

[-0.598186194896698, -0.13584637641906738, -0.888805091381073, -0.0012239335337653756, -0.6766712665557861, 0.46304798126220703, -0.14941516518592834, -0.8749025464057922, 0.36421218514442444, 0.5214415192604065, 0.35548847913742065, -0.8463447093963623, 0.5632082223892212, 0.5150635838508606, -0.951123058795929, 0.7784736156463623]